In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from xgboost.sklearn import XGBClassifier
import numpy as np 
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier as knn
from sklearn.ensemble import ExtraTreesClassifier as etc
from sklearn import svm
np.random.seed(0)

#Loading data
df_train = pd.read_csv('./air/joinedTrain.csv')
df_test = pd.read_csv('./air/joinedTest.csv')

labels = df_train['fault_severity'].values
df_train = df_train.drop(['fault_severity'], axis=1)
id_test = df_test['id']
piv_train = df_train.shape[0]

#Creating a DataFrame with train+test data
df_all = pd.concat((df_train, df_test), axis=0, ignore_index=True)
df_all = df_all.drop(['id'], axis=1)

#Filling nan
df_all = df_all.fillna(-1)

#####Feature engineering#######
#One-hot-encoding features
df_all = df_all.drop(['location'], axis=1)
ohe_feats = ['severity_type', 'locInt']#, 'location']
for f in ohe_feats:
    df_all_dummy = pd.get_dummies(df_all[f], prefix=f)
    df_all = df_all.drop([f], axis=1)
    df_all = pd.concat((df_all, df_all_dummy), axis=1)

#Splitting train and test
vals = df_all.values
X = vals[:piv_train]
le = LabelEncoder()
y = le.fit_transform(labels)   
X_test = vals[piv_train:]

#Classifier
xgb = XGBClassifier(max_depth=6, learning_rate=0.25, n_estimators=43,
                    objective='multi:softprob', subsample=0.6, colsample_bytree=0.6, seed=0)                  
#xgb = XGBClassifier(n_estimators=1)
xgb.fit(X, y)

y_pred = xgb.predict_proba(X_test)  

sub = pd.read_csv('./air/sample_submission.csv')
sub['predict_0'] = y_pred[:, 0]
sub['predict_1'] = y_pred[:, 1]
sub['predict_2'] = y_pred[:, 2]
    
#Generate submission
sub.to_csv('./air/res.csv',index=False)